In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.text import *
from pathlib import Path
import pandas as pd

In [3]:
#path = Path('/home/ubuntu/data/autopsy/milestone/no_preprocessing')

path = Path('/home/ubuntu/data/autopsy/2014_2018/no_preprocessing/reports/k-fold-suicide-homicide-accident-natural')


In [4]:
df = pd.read_csv(path/'data_suicide_homicide_accident_natural_k_1.csv')
df.head()

,label,text,is_valid
0,Accident,\n.MICRO.\nHEART: Scattered myofibers are enla...,False
1,Accident,\n.MICRO.\nHeart: negative\n\nLungs: mild emph...,False
2,Accident,\n.MICRO.\nNONE\n.LOCATIONHead\n.INJURYBlunt i...,False
3,Accident,\n.MICRO.\nBrain: No significant histopatholog...,False
4,Accident,"\n.MICRO.\nLIVER: Cirrhosis present, characte...",False


In [5]:
df['text'][1]

'\n.MICRO.\nHeart: negative\n\nLungs: mild emphysema; autolysis\n\nLiver: fatty change; autolysis\n\nKidney: autolysis\n\nThyroid: negative\n\nEsophagus: autolysis\n.LOCATIONExtremity\n.INJURYBlunt injury.\n.INJURYDESC.\nThere is an 8 x 4 cm area of discontinuous abrasions overlying the left knee.\n\n.LOCATIONBack\n.INJURYBlunt injury.\n.INJURYDESC.\nThere is a 4 x 2 cm irregular pressure abrasion overlying the lumbar spine.\n\nThere is a 3 x 1.5 cm pressure abrasion on the lower left buttock.'

In [6]:
bs=32

In [7]:
awd_lstm_lm_config = dict(emb_sz=400, n_hid=1150, n_layers=3, pad_token=1, qrnn=False, bidir=True, output_p=0.25,
                          hidden_p=0.1, input_p=0.2, embed_p=0.02, weight_p=0.15, tie_weights=True, out_bias=True)

awd_lstm_clas_config = dict(emb_sz=400, n_hid=1150, n_layers=3, pad_token=1, qrnn=False, bidir=True, output_p=0.4,
                       hidden_p=0.2, input_p=0.6, embed_p=0.1, weight_p=0.5)

In [8]:

classes = ['Suicide','Homicide','Accident','Natural']

In [9]:
valid_df =  df.loc[df['is_valid']==True]

In [10]:
train_df = df.loc[df['is_valid']==False]

In [ ]:
data_lm = TextLMDataBunch.from_df(path, train_df=train_df, valid_df=valid_df, classes=classes,bs=bs)

In [ ]:
data_lm.show_batch()

In [ ]:
learn = language_model_learner(data_lm, arch=AWD_LSTM, config=awd_lstm_lm_config, pretrained=False)

In [ ]:
learn.freeze()

In [ ]:
#do some gradual unfreezing, so we don't lose the pretrained info
learn.fit_one_cycle(2, 1e-2, moms=(0.8,0.7))

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2), moms=(0.8,0.7))

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3), moms=(0.8,0.7))

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(8, 1e-3, moms=(0.8,0.7))

In [ ]:
learn.save_encoder('fine_tuned_enc_no_pretrain2')

In [ ]:
data_clas = TextClasDataBunch.from_df(path, train_df=train_df, valid_df=valid_df, vocab=data_lm.train_ds.vocab, bs=bs, classes=classes)

In [ ]:
data_clas.show_batch()

In [ ]:
learn = text_classifier_learner(data_clas, arch=AWD_LSTM, drop_mult=0.8, config=awd_lstm_clas_config, pretrained=False)
learn.load_encoder('fine_tuned_enc_no_pretrain2')
learn.freeze()

In [ ]:
learn.fit_one_cycle(1, 2e-2, moms=(0.8,0.7))

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2), moms=(0.8,0.7))

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3), moms=(0.8,0.7))

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(20, slice(3e-4,3e-3), moms=(0.8,0.7))

In [ ]:
from fastai.vision import *

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(4,4), dpi=120)

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.recorder.plot_metrics()

In [ ]:
def intrinsic_attention_modified_old(self, text:str, class_id:int=None):
        ids = self.data.one_item(text)[0]
        emb = self.model[0].module.encoder(ids).detach().requires_grad_(True)
        self.model.train()
        self.model.zero_grad()
        self.model.reset()
        lstm_output = self.model[0].module(emb, from_embeddings=True)
        self.model.eval()
        self.model.zero_grad()
        self.model.reset()
        cl = self.model[1](lstm_output)[0].softmax(dim=-1)
        if class_id is None: class_id = cl.argmax()
        cl[0][class_id].backward()
        attn = emb.grad.squeeze().abs().sum(dim=-1)
        attn /= attn.max()
        tokens = self.data.single_ds.reconstruct(ids[0])
        return tokens, attn
    
def intrinsic_attention_modified(self, text:str, class_id:int=None):
        """Calculate the intrinsic attention of the input w.r.t to an output `class_id`, or the classification given by the model if `None`.
        For reference, see the Sequential Jacobian session at https://www.cs.toronto.edu/~graves/preprint.pdf
        """
        self.model.train()
        eval_dropouts(self.model)
        self.model.zero_grad()
        self.model.reset()
        ids = self.data.one_item(text)[0]
        emb = self.model[0].module.encoder(ids).detach().requires_grad_(True)                
        lstm_output = self.model[0].module(emb, from_embeddings=True)
        self.model.eval()
        cl = self.model[1](lstm_output)[0].softmax(dim=-1)
        if class_id is None: class_id = cl.argmax()
        cl[0][class_id].backward()
        attn = emb.grad.squeeze().abs().sum(dim=-1)
        attn /= attn.max() 
        tokens = self.data.single_ds.reconstruct(ids[0])
        return tokens, attn

def eval_dropouts(mod):
        module_name =  mod.__class__.__name__
        if 'Dropout' in module_name or 'BatchNorm' in module_name: mod.training = False
        for module in mod.children(): eval_dropouts(module)

TextClassificationInterpretation.intrinsic_attention = intrinsic_attention_modified
ci = TextClassificationInterpretation.from_learner(learn)

In [ ]:
import matplotlib.cm as cm

In [ ]:
def interpret_result(row_index):
    example = df['text'][row_index]
    label = df['label'][row_index]
    print('True Label:',label)
    
    prediction_class = learn.predict(example)[0]
    prediction_probs = learn.predict(example)[2]
    
    print('\nPred Label:',prediction_class)
    print('\nClass Probabilities:',prediction_probs)
    
    print('\n***Raw Text***')
    print(example)
    
    print('\n***Intrinsic Attention***')
    
    ci.show_intrinsic_attention(example,  cmap=cm.Reds)
    
    prediction_class = learn.predict(example)[0]
    prediction_probs = learn.predict(example)[2]
    
    
    

In [ ]:
interpret_result(13)

In [ ]:
interpret_result(20)

In [ ]:
interpret_result(107)

In [ ]:
interpret_result(117)

In [ ]:
interpret_result(134)

In [ ]:
interpret_result(138)

In [ ]:
interpret_result(10248)

In [ ]:
interpret_result(10252)

In [ ]:
interpret_result(10307)

In [ ]:
interpret_result(10186)

In [ ]:
interpret_result(10279)

In [ ]:
valid = df.loc[df['is_valid']==True]

In [ ]:
valid

In [ ]:
'''for i in range(0,3040):
    example = df['text'][i]
    label = df['label'][i]
    val = df['is_valid'][i]
    prediction_class = learn.predict(example)[0]
    if str(label) != str(prediction_class):
        print(i, label, prediction_class, val)
'''